In [1]:
import cdsapi
import xarray as xr
import time
import rioxarray as rio

In [2]:
import sys
sys.path.append('..')

In [3]:
import utils

In [ ]:
import datetime

utils.download_data(
    variable = '2m_temperature',
    daily_statistic = 'daily_mean',
    date = datetime.datetime(2020, 2, 1),
    download_filepath = '../data/2mtemp_2021_02.nc',
    exist_ok = True,
    request_full_month = True,
)

In [ ]:
t2m = utils.load_mean_temperature_nc_file(
    nc_filepath = '../data/2mtemp_2021_02.nc',
)

type(t2m)

In [ ]:
t2m

In [7]:
import geopandas as gpd

malawi_gdf = gpd.read_file('/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/data/malawi/mwi_adm_nso_hotosm_20230405_shp/mwi_admbnda_adm0_nso_hotosm_20230405.shp')

In [ ]:
malawi_gdf.crs

In [ ]:
malawi_gdf.envelope.values

In [24]:
clipped_t2m = t2m.rio.clip(malawi_gdf.envelope.values, malawi_gdf.crs, drop=True, all_touched=True)

In [ ]:
t2m.shape

In [ ]:
clipped_t2m.shape

In [27]:
clipped_t2m.rio.to_raster(
    "../data/malawi_2mtemp_2021_02.tif",
    tiled=True,
    compress="lzw",
)

In [ ]:
startmonth = 5

[(startmonth + i) % 12 + 1 for i in range(24)]

In [32]:
def get_dates(
    startdate:datetime.datetime,
    enddate:datetime.datetime,
):
    startyear = startdate.year
    startmonth = startdate.month
    endyear = enddate.year
    endmonth = enddate.month

    dates = []
    
    for year in range(startyear, endyear + 1):
        month = startmonth if year == startyear else 1
        stopmonth = endmonth if year == endyear else 12
        while month <= stopmonth:
            dates.append(datetime.datetime(year, month, 1))
            month += 1

    return dates

In [ ]:
get_dates(startdate=datetime.datetime(2020, 6, 12), enddate=datetime.datetime(2022, 4, 27))